In [ ]:
for i in range(1,4):
    for grd in range(1,9):
        for unidad in ["ICU", "SDU_WARD"]:
            # ---- 1. Filter your dataset
            tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
            v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
            # v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

            # ---- 2. Get LOS value counts
            vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
            vector.columns = ["LOS", "count"]

            # ---- 3. Build raw data from LOS values and their frequencies
            los = np.array(vector["LOS"])
            ocurrencias = np.array(vector["count"])
            raw_data = np.repeat(los, ocurrencias)


            # ---- 4. Definir los bins de 12 horas
            bin_width = 12
            max_los = np.max(raw_data)
            edges = np.arange(0, max_los + bin_width, bin_width)
            midpoints = (edges[:-1] + edges[1:]) / 2  # midpoint of (a, b] is (a + b) / 2
    

            # ---- 5. Meter datos a bins individuales (0 - 12], (12 - 24], ...)
            # agrega 0 si no hay datos para ese bin
            bin_indices = np.digitize(raw_data, edges, right=True)
            hist = np.array([(bin_indices == i).sum() for i in range(1, len(edges))])

#--------------------------------Hasta aca esta bien----------------------------------------

            # ---- 6. Fit log-normal distribution to the raw data
            shape, loc, scale = stats.lognorm.fit(raw_data, floc=0)



            # # ---- Optimización
            # def chi2_objective(params, hist, edges, total_count):
            #     shape, loc, scale = params
            #     # Ensure parameters are valid
            #     if shape <= 0 or scale <= 0:
            #         return np.inf
            #     cdf_low = stats.lognorm.cdf(edges[:-1], shape, loc, scale)
            #     cdf_high = stats.lognorm.cdf(edges[1:], shape, loc, scale)
            #     expected_probs = cdf_high - cdf_low
            #     expected_counts = expected_probs * total_count

            #     # Mask for bins with enough expected values
            #     mask = expected_counts >= 5
            #     if not np.any(mask):
            #         return np.inf
            #     obs = hist[mask]
            #     exp = expected_counts[mask]
                
            #     # Normalize expected to match sum of obs
            #     exp *= obs.sum() / exp.sum()

            #     chi2 = np.sum((obs - exp)**2 / exp)
            #     return chi2
            # initial_shape, initial_loc, initial_scale = stats.lognorm.fit(raw_data, floc=0)
            # # Minimize the chi-square
            # result = minimize(
            #     chi2_objective,
            #     x0=[initial_shape, initial_loc, initial_scale],
            #     args=(hist, edges, len(raw_data)),
            #     bounds=[(1e-5, None), (0, None), (1e-5, None)],
            #     method='L-BFGS-B'
            # )
            # # Extract optimized parameters
            # opt_shape, opt_loc, opt_scale = result.x
            # # print(f"Optimized shape={opt_shape:.4f}, loc={opt_loc:.4f}, scale={opt_scale:.2f}")
            # shape = opt_shape
            # loc = opt_loc
            # scale = opt_scale


            # # ---- 7. Compute expected counts per bin using log-normal CDF
            # cdf_low = stats.lognorm.cdf(edges[:-1], shape, loc, scale)
            # cdf_high = stats.lognorm.cdf(edges[1:], shape, loc, scale)
            # expected_probs = cdf_high - cdf_low
            # # Acumulo todo lo que queda de la cola en el ultimo bin
            # # expected_probs[-1] = 1 - (sum(expected_probs) - expected_probs[-1])
            # expected_counts = expected_probs * len(raw_data)


            # # ---- 9. Chi-square test (only where expected counts ≥ 5)
            # mask = expected_counts >= 5
            # obs = hist[mask]
            # exp = expected_counts[mask]

            # # display(obs)
            # # display(exp)

            # # Normalize expected counts to match the total of observed
            # if exp.sum() > 0:
            #     exp *= obs.sum() / exp.sum()

            # # Chi-square test
            # if np.any(mask):
            #     chi2, p = stats.chisquare(f_obs=obs, f_exp=exp)
            #     if p > 0.05:
            #         print(f"Chi² = {chi2:.2f}, p-value = {p:.10f}, Unidad={unidad}, Hospital={i}, GRD={grd}")
                    
            #         # ---- 8. Plot observed vs. expected
            #         plt.figure(figsize=(12, 6))
            #         plt.bar(midpoints, hist, width=bin_width - 2, alpha=0.6, label='Observed (12h bins)')
            #         plt.plot(midpoints, expected_counts, 'r--o', label='Log-normal expected')
            #         plt.xlabel('Length of Stay (hours)')
            #         plt.ylabel('Frequency')
            #         plt.title(f'LOS Histogram vs Log-normal Fit\nUnidad={unidad}, Hospital={i}, GRD={grd}')
            #         plt.grid(True)

            #         # Curva continua
            #         x_vals = np.linspace(1, max_los, 500)
            #         pdf_vals = stats.lognorm.pdf(x_vals, s=shape, loc=loc, scale=scale)
            #         pdf_scaled = pdf_vals * len(raw_data) * bin_width
            #         plt.plot(x_vals, pdf_scaled, 'g-', label='Log-normal PDF (smoothed)')

            #         plt.legend()
            #         plt.show()
            #         break

                    
            # else:
            #     print("Chi-square test skipped: not enough expected counts ≥ 5.")


In [ ]:
# Tabla para los OR

tabla = {
    1: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    2: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    3: [["MS_GRD", "LOS = 12", "LOS = 24"]]
}

for i in range(1,4):
    for grd in range(1,9):
        for unidad in ["OR"]:
            # ---- 1. Filter your dataset
            tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
            v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
            # v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

            # ---- 2. Get LOS value counts
            vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
            vector["%"] = vector["count"] / vector["count"].sum()
            vector1 = vector[["LOS", "%"]].reset_index()

            los_12 = 0
            los_24 = 0
            for index, row in vector1.iterrows():
                if row["LOS"] == 12:
                    los_12 = row["%"]
                elif row["LOS"] == 24:
                    los_24 = row["%"]

            tabla[i].append([grd, round(los_12,5), round(los_24,5)])
                    
            

    display(tabla[i])



In [ ]:
# Pasar a tablas de latex
for key, h in tabla.items():

    texto = f"""
\\begin{{table}}[H]
    \\centering
    \\begin{{tabular}}{{ccc}}
        \\toprule
        MS\_GRD & LOS = 12 & LOS = 24 \\\\
        \\midrule
        {h[1][0]} & {h[1][1]} & {h[1][2]} \\\\
        {h[2][0]} & {h[2][1]} & {h[2][2]} \\\\
        {h[3][0]} & {h[3][1]} & {h[3][2]} \\\\
        {h[4][0]} & {h[4][1]} & {h[4][2]} \\\\
        {h[5][0]} & {h[5][1]} & {h[5][2]} \\\\
        {h[6][0]} & {h[6][1]} & {h[6][2]} \\\\
        {h[7][0]} & {h[7][1]} & {h[7][2]} \\\\
        {h[8][0]} & {h[8][1]} & {h[8][2]} \\\\
        \\bottomrule
    \\end{{tabular}}
    \\caption{{Probabilidad LOS en OR para Hospital: {key}}}
    \\label{{tab:Probabilidad LOS en OR para Hospital: {key}}}
\\end{{table}}
    """

    print(texto)

In [ ]:
# Segundo intento (quedo pesimo)

# for i in range(1,4):
# for grd in range(1,9):
# for unidad in ["ICU", "SDU_WARD"]:

unidad = "ICU"  # Cambiar unidad
grd = 1

# ---- 1. Filtra los datos para obtener LOS y cantidad de ocurrencias
tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
# v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

# ---- 2. Calcular la cantidad de veces que se repite ese LOS
vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
vector.columns = ["LOS", "count"]

# ---- 3. Pasarlo a formato para histograma
los = np.array(vector["LOS"])
ocurrencias = np.array(vector["count"])
raw_data = np.repeat(los, ocurrencias) # multiplica LOS por cantidad de repeticiones

# ---- 4. Definir los bins de 12 horas (para ajustar la curva)
bin_width = 12
max_los = np.max(raw_data) # Elige el maximo de LOS
edges = np.arange(0, max_los + bin_width, bin_width) # Define los bins
midpoints = (edges[:-1] + edges[1:]) / 2  # Define puntos medios de los bins

# ---- 5. Meter datos a bins individuales (0 - 12], (12 - 24], ...) agrega 0 si no hay datos para ese bin
bin_indices = np.digitize(raw_data, edges, right=True)
hist = np.array([(bin_indices == i).sum() for i in range(1, len(edges))])
bin_edges = edges
bin_upper_bounds = bin_edges[1:]

# ---- 6. Ajustar distribución a los datos en bins de 12 horas
shape, loc, scale = stats.lognorm.fit(raw_data, floc = 0)

# # Grafico para la curva suave log_normal
# x = np.linspace(min(bin_upper_bounds), max(bin_upper_bounds), 1000)
# y = stats.lognorm.pdf(x, shape, loc, scale) * len(raw_data) * bin_width
# plt.hist(raw_data, bins=bin_edges, alpha=0.6, color='g', label="Histogram")
# plt.plot(x, y, label="Fitted Log-normal", color='r')
# plt.xlabel('Value')
# plt.ylabel('Density')
# plt.legend()
# plt.show()

# Define interval of interest
  # for example
N = len(raw_data)
maximo = int(raw_data[-1])
maximo = int(maximo/12)
expected = []
for i in range(0, maximo):
    a = i * 12
    b = (i + 1) * 12
    p_interval = stats.lognorm.cdf(b, shape, loc, scale) - stats.lognorm.cdf(a, shape, loc, scale)
    expected_occurrences = N * p_interval
    expected.append(expected_occurrences)
esperado = np.array(expected)
observado = hist

# Test de chi cuadrado
mask = esperado >= 5

obs = observado[mask]
exp = esperado[mask]

exp *= obs.sum() / exp.sum()

chi2, p = stats.chisquare(f_obs=obs, f_exp=exp)
print(f"chi2 = {chi2}, p = {p}")